In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/u/brg8ve/.local/src/counseling-feedback/unsloth/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA RTX 4000 Ada Generation. Num GPUs = 4. Max memory: 19.559 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.11.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [117]:
from datasets import load_dataset

dataset = load_dataset("avylor/feedback_qesconv", split="train")
print(dataset.column_names)

['conv_index', 'helper_index', 'text']


In [118]:
from unsloth import get_chat_template

In [119]:
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

In [123]:
def convert_to_hf_format(example):
    tmp = example["text"].rsplit("\n\n" )
    system = tmp[0]
    user = tmp[1]+"\n\n"+tmp[2]
    assistant = tmp[3]
    return {"text": [
        {"role":"system", "content":system},
        {"role":"user", "content":user},
        {"role":"assistant", "content":assistant},
            ]}

In [124]:
dataset

Dataset({
    features: ['conv_index', 'helper_index', 'text'],
    num_rows: 8179
})

In [125]:
dataset[0]

{'conv_index': 142,
 'helper_index': 7,
 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGive feedback to the Helper\'s last response.\n\n### Input:\nSeeker: yha. also, im worried i might fail again if i get too distracted by my family\nHelper: Is it possible for you to get any sort of private tutoring which might help you?\nSeeker: i mean, tutoring is just another added time commitment. I really failed the first time because i was depressed and unmotivated. which is why I\'m excited to be with my family finally\nHelper: Ah .. in that case is it possible to talk to your doctor? They might be able to get a special allowance for you given your personal circumstances\nSeeker: yeah. that\'s another thing. I havent told my therapist or even my parents yet. that i failed\nHelper: I think that it would be a really good idea if you talked to them about it.

In [126]:
dataset = dataset.map(convert_to_hf_format, remove_columns=dataset.features)

Map: 100%|██████████| 8179/8179 [00:00<00:00, 15566.33 examples/s]


In [127]:
dataset

Dataset({
    features: ['text'],
    num_rows: 8179
})

In [128]:
dataset[0]

{'text': [{'content': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.',
   'role': 'system'},
  {'content': "### Instruction:\nGive feedback to the Helper's last response.\n\n### Input:\nSeeker: yha. also, im worried i might fail again if i get too distracted by my family\nHelper: Is it possible for you to get any sort of private tutoring which might help you?\nSeeker: i mean, tutoring is just another added time commitment. I really failed the first time because i was depressed and unmotivated. which is why I'm excited to be with my family finally\nHelper: Ah .. in that case is it possible to talk to your doctor? They might be able to get a special allowance for you given your personal circumstances\nSeeker: yeah. that's another thing. I havent told my therapist or even my parents yet. that i failed\nHelper: I think that it would be a really good idea if you talked to them abo

In [129]:
def formatting_prompts_func(examples):
    convos = examples["text"]
    texts = tokenizer.apply_chat_template(convos, tokenize = False, add_generation_prompt = False)
    return { "text" : texts, }

In [130]:
tokenizer.apply_chat_template(dataset[0]["text"], tokenize=False, add_generation_prompt = False)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n### Instruction:\nGive feedback to the Helper\'s last response.\n\n### Input:\nSeeker: yha. also, im worried i might fail again if i get too distracted by my family\nHelper: Is it possible for you to get any sort of private tutoring which might help you?\nSeeker: i mean, tutoring is just another added time commitment. I really failed the first time because i was depressed and unmotivated. which is why I\'m excited to be with my family finally\nHelper: Ah .. in that case is it possible to talk to your doctor? They might be able to get a special allowance for you given your personal circumstances\nSeeker: yeah. that\'s another thing. I havent told my t

In [131]:
formatting_prompts_func(dataset[0])

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n### Instruction:\nGive feedback to the Helper\'s last response.\n\n### Input:\nSeeker: yha. also, im worried i might fail again if i get too distracted by my family\nHelper: Is it possible for you to get any sort of private tutoring which might help you?\nSeeker: i mean, tutoring is just another added time commitment. I really failed the first time because i was depressed and unmotivated. which is why I\'m excited to be with my family finally\nHelper: Ah .. in that case is it possible to talk to your doctor? They might be able to get a special allowance for you given your personal circumstances\nSeeker: yeah. that\'s another thing. I havent 

In [132]:
dataset = dataset.map(formatting_prompts_func, remove_columns=dataset.features)

Map:   0%|          | 0/8179 [00:00<?, ? examples/s]

Map: 100%|██████████| 8179/8179 [00:01<00:00, 6884.28 examples/s]


In [133]:
dataset

Dataset({
    features: ['text'],
    num_rows: 8179
})

In [134]:
dataset[0]

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n### Instruction:\nGive feedback to the Helper\'s last response.\n\n### Input:\nSeeker: yha. also, im worried i might fail again if i get too distracted by my family\nHelper: Is it possible for you to get any sort of private tutoring which might help you?\nSeeker: i mean, tutoring is just another added time commitment. I really failed the first time because i was depressed and unmotivated. which is why I\'m excited to be with my family finally\nHelper: Ah .. in that case is it possible to talk to your doctor? They might be able to get a special allowance for you given your personal circumstances\nSeeker: yeah. that\'s another thing. I havent 

In [162]:
from trl import SFTConfig, SFTTrainer
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #max_steps = 60,
        num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

In [163]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [164]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX 4000 Ada Generation. Max memory = 19.559 GB.
2.947 GB of memory reserved.


In [165]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,179 | Num Epochs = 1 | Total steps = 1,023
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Step,Training Loss
1,0.900200
2,0.704500
3,0.809400
4,1.030000
5,0.604500
6,0.743900
7,0.821600
8,0.742400
9,0.972100
10,0.901600


In [167]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Fibonacci sequence is 1, 1, 2, 3, 5, 8, 13, and so on. Based on the given sequence: 1, 1, 2, 3, 5, 8, 13, 21, the next number in the sequence is: 21<|eot_id|>


In [168]:
model.save_pretrained("llama32_sft")
tokenizer.save_pretrained("llama32_sft")

('llama32_sft/tokenizer_config.json',
 'llama32_sft/special_tokens_map.json',
 'llama32_sft/chat_template.jinja',
 'llama32_sft/tokenizer.json')